# Arabic GEC Model - Test Evaluation

This notebook evaluates the trained AraT5 GEC model on the QALB 2014 test set.

**What it does:**
1. Downloads the QALB 2014 test dataset
2. Downloads your trained model from Google Drive
3. Runs evaluation and calculates BLEU & GLEU scores for your research paper



## Step 1: Install Dependencies

In [1]:
!pip install -q transformers datasets gdown sentencepiece evaluate sacrebleu tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 6.8 MB/s eta 0:00:00


## Step 2: Download Test Dataset (QALB 2014)

In [2]:
import gdown
import zipfile
import os

# Download the QALB dataset from Google Drive
file_id = '1hvLiiMvvubyCEAZK4KIWgu7qHBNCHOp-'
url = f'https://drive.google.com/uc?id={file_id}'
output_file = 'qalb_dataset.zip'

if not os.path.exists(output_file):
    print("📥 Downloading QALB dataset...")
    gdown.download(url, output_file, quiet=False)
    print("✅ Download complete!")
else:
    print("✅ Dataset already downloaded")

# Extract the test files (the actual path includes 'data' folder)
if not os.path.exists("QALB-0.9.1-Dec03-2021-SharedTasks/data/2014/test"):
    print("📦 Extracting test data...")
    with zipfile.ZipFile(output_file, 'r') as zip_ref:
        # Extract only the test folder to save space
        for member in zip_ref.namelist():
            if 'data/2014/test/' in member or '2014/test/' in member:
                zip_ref.extract(member, '.')
    print("✅ Test data extracted!")
else:
    print("✅ Test data already extracted")

📥 Downloading QALB dataset...


Downloading...
From (original): https://drive.google.com/uc?id=1hvLiiMvvubyCEAZK4KIWgu7qHBNCHOp-
From (redirected): https://drive.google.com/uc?id=1hvLiiMvvubyCEAZK4KIWgu7qHBNCHOp-&confirm=t&uuid=8f3d0363-3dcb-49cc-a2f6-4e2cf4a9fe3a
To: /content/qalb_dataset.zip
100%|██████████| 94.3M/94.3M [00:02<00:00, 35.3MB/s]


✅ Download complete!
📦 Extracting test data...
✅ Test data extracted!


## Step 3: Download Trained Model from Google Drive

In [3]:
import gdown
import os

# Your trained model folder on Google Drive
model_folder_id = "17q2WKD-Bu_2nK_vuhNEpKBWMOeMX3tHI"
model_folder_url = f"https://drive.google.com/drive/folders/{model_folder_id}"

if os.path.exists("./arat5_gec_model_output"):
    print("✅ Model already downloaded")
else:
    print("📥 Downloading trained model from Google Drive...")
    print(f"   Source: {model_folder_url}")

    try:
        gdown.download_folder(model_folder_url, output=".", quiet=False, use_cookies=False)
        print("✅ Model downloaded successfully!")
    except Exception as e:
        print(f"❌ Download failed: {e}")
        print("\n💡 Please ensure the folder is set to 'Anyone with the link can view'")

Retrieving folder contents


📥 Downloading trained model from Google Drive...
   Source: https://drive.google.com/drive/folders/17q2WKD-Bu_2nK_vuhNEpKBWMOeMX3tHI
Retrieving folder 1IXj7O2z3LIhgf7t4e1axXXLdq8CzhPeU arat5_gec_model_output
Processing file 1R3z8Z1wD2ntqih5d2ehgrQJxaqBtTNO4 config.json
Processing file 1BZCqkXMctEDGDQ4Whh9SYVdoNzLHsoFb generation_config.json
Processing file 13xkm3_hr2_miQ9hezbqVRjsPR_XuMVg_ model.safetensors
Processing file 1ABdDRls366kf9NUpDWfnZTnMd5qSL0BH special_tokens_map.json
Processing file 1P7FZb3Lct-Vo0wfeg5vlIphsZVxn6T8x spiece.model
Processing file 1b0x6oN7AJ4CmqVMROVjTqgt1bVWG4hY7 tokenizer_config.json
Processing file 1eDuSAkfTl-VDG0_9OOUQqIZSsUxb_MpN tokenizer.json


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1R3z8Z1wD2ntqih5d2ehgrQJxaqBtTNO4
To: /content/arat5_gec_model_output/config.json
100%|██████████| 809/809 [00:00<00:00, 1.04MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BZCqkXMctEDGDQ4Whh9SYVdoNzLHsoFb
To: /content/arat5_gec_model_output/generation_config.json
100%|██████████| 122/122 [00:00<00:00, 82.1kB/s]
Downloading...
From (original): https://drive.google.com/uc?id=13xkm3_hr2_miQ9hezbqVRjsPR_XuMVg_
From (redirected): https://drive.google.com/uc?id=13xkm3_hr2_miQ9hezbqVRjsPR_XuMVg_&confirm=t&uuid=7810970a-8cfa-46de-b127-371ed1b1be48
To: /content/arat5_gec_model_output/model.safetensors
100%|██████████| 1.47G/1.47G [00:29<00:00, 50.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ABdDRls366kf9NUpDWfnZTnMd5qSL0BH
To: /content/arat5_gec_model_output/special_tokens_map.json
100%|██████████| 2.54k/2.54k [00

✅ Model downloaded successfully!



Download completed


## Step 4: Run Evaluation (BLEU & GLEU)

This cell:
- Loads your trained model
- Generates corrections for all test sentences
- Calculates BLEU and GLEU scores
- Saves predictions to a file

**Expected runtime:** 5-10 minutes on GPU

In [4]:
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm import tqdm
import evaluate

# ===== Configuration =====
model_path = "./arat5_gec_model_output"

# Find test data (handle different possible locations)
test_sent_path = None
test_cor_path = None

possible_paths = [
    "./QALB-0.9.1-Dec03-2021-SharedTasks/data/2014/test/QALB-2014-L1-Test",
    "./2014/test/QALB-2014-L1-Test",
    "./QALB-0.9.1-Dec03-2021-SharedTasks/2014/test/QALB-2014-L1-Test"
]

for base_path in possible_paths:
    if os.path.exists(f"{base_path}.sent") and os.path.exists(f"{base_path}.cor"):
        test_sent_path = f"{base_path}.sent"
        test_cor_path = f"{base_path}.cor"
        break

# ===== Verify Files =====
if not os.path.exists(model_path):
    print(f"❌ Model not found at '{model_path}'. Please run Step 3 first.")
    raise FileNotFoundError("Model not found")

if not test_sent_path:
    print("❌ Test files not found. Please run Step 2 first.")
    raise FileNotFoundError("Test data not found")

print(f"✅ Model path: {model_path}")
print(f"✅ Test data: {test_sent_path}")

# ===== Load Model & Tokenizer =====
print("\n🔄 Loading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()
print(f"✅ Model loaded on {device}")

# ===== Load Test Data =====
print("\n📖 Loading test data...")
with open(test_sent_path, 'r', encoding='utf-8') as f:
    src_sentences = [line.strip() for line in f.readlines()]

with open(test_cor_path, 'r', encoding='utf-8') as f:
    # Fix: Remove "S " prefix if present in reference sentences
    ref_lines = f.readlines()
    ref_sentences = []
    for line in ref_lines:
        clean_line = line.strip()
        if clean_line.startswith("S "):
            clean_line = clean_line[2:].strip()
        ref_sentences.append(clean_line)

print(f"✅ Loaded {len(src_sentences)} test sentences")

# ===== Generate Predictions =====
print("\n🚀 Generating predictions (this may take 5-10 minutes)...")
predictions = []
batch_size = 16
prefix = "gec_arabic: "

for i in tqdm(range(0, len(src_sentences), batch_size)):
    batch = src_sentences[i:i + batch_size]
    batch_with_prefix = [prefix + sent for sent in batch]

    inputs = tokenizer(batch_with_prefix, return_tensors="pt", padding=True, truncation=True, max_length=1024)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=1024, num_beams=5)

    batch_predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    predictions.extend(batch_predictions)

print(f"✅ Generated {len(predictions)} predictions")

# ===== Calculate BLEU Score =====
print("\n📊 Calculating BLEU score...")
bleu = evaluate.load("sacrebleu")
bleu_result = bleu.compute(predictions=predictions, references=[[r] for r in ref_sentences])
bleu_score = bleu_result['score']

# ===== Calculate GLEU Score =====
print("📊 Calculating GLEU score...")
gleu = evaluate.load("google_bleu")
gleu_result = gleu.compute(predictions=predictions, references=[[r] for r in ref_sentences])
gleu_score = gleu_result['google_bleu'] * 100  # Convert to percentage

# ===== Display Results =====
print("\n" + "="*60)
print("📈 EVALUATION RESULTS")
print("="*60)
print(f"BLEU Score:  {bleu_score:.2f}")
print(f"GLEU Score:  {gleu_score:.2f}")
print("="*60)

# ===== Save Predictions =====
output_file = "qalb_test_predictions.txt"
print(f"\n💾 Saving predictions to '{output_file}'...")
with open(output_file, 'w', encoding='utf-8') as f:
    for src, pred, ref in zip(src_sentences, predictions, ref_sentences):
        f.write(f"Source:     {src}\n")
        f.write(f"Predicted:  {pred}\n")
        f.write(f"Reference:  {ref}\n")
        f.write("-"*30 + "\n")

print(f"\n✅ Predictions saved to '{output_file}'")
print("\n⚠️ You can now use these scores in your research paper!")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


✅ Model path: ./arat5_gec_model_output
✅ Test data: ./QALB-0.9.1-Dec03-2021-SharedTasks/data/2014/test/QALB-2014-L1-Test.sent

🔄 Loading model and tokenizer...
✅ Model loaded on cuda

📖 Loading test data...
✅ Loaded 968 test sentences

🚀 Generating predictions (this may take 5-10 minutes)...


100%|██████████| 61/61 [05:30<00:00,  5.41s/it]


✅ Generated 968 predictions

📊 Calculating BLEU score...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


📊 Calculating GLEU score...



📈 EVALUATION RESULTS
BLEU Score:  72.33
GLEU Score:  71.52

💾 Saving predictions to 'qalb_test_predictions.txt'...

✅ Predictions saved to 'qalb_test_predictions.txt'

⚠️ You can now use these scores in your research paper!


## Step 5: (Optional) View Sample Predictions

In [5]:
# Show 5 random examples
import random

print("\n📋 Sample Predictions:\n")
print("-" * 80)

indices = random.sample(range(len(src_sentences)), min(5, len(src_sentences)))

for idx in indices:
    print(f"\n[Example {idx+1}]")
    print(f"Input:      {src_sentences[idx]}")
    print(f"Reference:  {ref_sentences[idx]}")
    print(f"Prediction: {predictions[idx]}")
    print("-" * 80)


📋 Sample Predictions:

--------------------------------------------------------------------------------

[Example 378]
Input:      aj_0003163_0000005.ar يلي براقب الإعلام السوري ، كان عم يحاول يغطي - بشكل مفاجئ - الثورة المصرية و الليبية كتير آخر فترة ، و يركز على الخلاف بين الإسلاميين و العلمانيين بمصر بس بعد قرار سحب السفير . صرت تشوف جماعة الأمن يلي بعلقو على المواقع الإخبارية عم يقولو الثورة المصرية ثورة مثقفين و الثورة السورية ثورة أميين و مخربين .
Reference:  الذي يراقب الإعلام السوري ، كان يحاول يغطي - بشكل مفاجئ - الثورة المصرية والليبية كثيرا آخر فترة ، ويركز على الخلاف بين الإسلاميين والعلمانيين بمصر بس بعد قرار سحب السفير . صرت تشوف جماعة الأمن الذين يعلقون على المواقع الإخبارية عم يقولون : الثورة المصرية ثورة مثقفين ، والثورة السورية ثورة أميين ومخربين .
Prediction: الذي يراقب الإعلام السوري ، كان يحاول يغطي - بشكل مفاجئ - الثورة المصرية والليبية كثيرا آخر فترة ، ويركز على الخلاف بين الإسلاميين والعلمانيين بمصر ، بس بعد قرار سحب السفير . صرت تشوف جماعة الأمن الذين يعلقون ع

## Step 6: Post-Processing - Arabic Tashkeel (Diacritization)
Now that we have the grammatically corrected sentences, we can apply an additional layer of "Tashkeel" (Diacritization) to show the full beauty of the corrected Arabic text.

We use the efficient **`Abdou/arabic-tashkeel-flan-t5-small`** model for this purpose.

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import random
import re

# Load a specialized model for Arabic Diacritization (Tashkeel)
tashkeel_model_name = "Abdou/arabic-tashkeel-flan-t5-small"
print(f"🔄 Loading Tashkeel model: {tashkeel_model_name}...")

tashkeel_tokenizer = AutoTokenizer.from_pretrained(tashkeel_model_name)
tashkeel_model = AutoModelForSeq2SeqLM.from_pretrained(tashkeel_model_name).to(device)

def apply_tashkeel(text):
    inputs = tashkeel_tokenizer(text, return_tensors="pt", max_length=256, truncation=True).to(device)
    with torch.no_grad():
        outputs = tashkeel_model.generate(**inputs, max_length=256)
    return tashkeel_tokenizer.decode(outputs[0], skip_special_tokens=True)

def strip_file_id(text):
    # Regex to match file IDs like "aj_0003074_0000029.ar" followed by a space
    # It looks for alphanumeric strings with underscores ending in .ar or .en
    clean_text = re.sub(r'^[a-z0-9_]+\.(ar|en)\s+', '', text)
    return clean_text

print("✅ Tashkeel model loaded!")

# Ensure we have indices to show
if 'indices' not in locals():
    indices = random.sample(range(len(src_sentences)), min(5, len(src_sentences)))

print("\n✨ GEC Predictions with Tashkeel (Diacritics):\n")
print("-" * 80)

for idx in indices:
    gec_pred = predictions[idx] # From the GEC model
    diacritized_pred = apply_tashkeel(gec_pred)

    # Strip ID for cleaner display
    original_input = strip_file_id(src_sentences[idx])

    print(f"\n[Example {idx+1}]")
    print(f"Original Input:     {original_input}")
    print(f"GEC Prediction:     {gec_pred}")
    print(f"With Tashkeel:      {diacritized_pred}")
    print("-" * 80)

🔄 Loading Tashkeel model: Abdou/arabic-tashkeel-flan-t5-small...
✅ Tashkeel model loaded!

✨ GEC Predictions with Tashkeel (Diacritics):

--------------------------------------------------------------------------------

[Example 378]
Original Input:     يلي براقب الإعلام السوري ، كان عم يحاول يغطي - بشكل مفاجئ - الثورة المصرية و الليبية كتير آخر فترة ، و يركز على الخلاف بين الإسلاميين و العلمانيين بمصر بس بعد قرار سحب السفير . صرت تشوف جماعة الأمن يلي بعلقو على المواقع الإخبارية عم يقولو الثورة المصرية ثورة مثقفين و الثورة السورية ثورة أميين و مخربين .
GEC Prediction:     الذي يراقب الإعلام السوري ، كان يحاول يغطي - بشكل مفاجئ - الثورة المصرية والليبية كثيرا آخر فترة ، ويركز على الخلاف بين الإسلاميين والعلمانيين بمصر ، بس بعد قرار سحب السفير . صرت تشوف جماعة الأمن الذين يعلقون على المواقع الإخبارية عم يقولوا الثورة المصرية ثورة مثقفين والثورة الثورة السورية ثورة أميين ومخربين .
With Tashkeel:      الَّذِي يُرَاقِبُ الإِعْلَامَ السُّورِيَّ ، كَانَ يُحَاوِلُ يُغَطِّي - بِشَكْلٍ مُفَاجِئٍ